In [2]:
from Bio import Entrez, SeqIO
import time
import csv
import re
import os
from collections import defaultdict

# ตั้งค่าอีเมลและ API Key
Entrez.email = "ounjai.sa@gmail.com"
Entrez.api_key = "573465c00e14b785945465d1dbe40c32ad08"

# =============================================================================
# การตั้งค่าการดึงข้อมูล
# =============================================================================

# ตั้งค่าไฟล์อินพุตและเอาต์พุต
FISH_LIST_FILE = "/Users/sarawut/Desktop/oneclick/thai_fish_database.txt"
OUTPUT_FASTA = "/Users/sarawut/Desktop/oneclick/thai_fish_sequences_ON_COI.fasta"
OUTPUT_METADATA = "/Users/sarawut/Desktop/oneclick/thai_fish_metadata_ON_COI.csv"

# ตั้งค่าสปีชีส์หลัก (จะดึงข้อมูลทั้งหมดที่หาได้)
TARGET_SPECIES = [
    "oreochromis niloticus",
    # เพิ่มสปีชีส์หลักอื่นๆ ได้ที่นี่
]

# ตั้งค่าจำนวนซีเควนซ์สำหรับสปีชีส์เปรียบเทียบ
COMPARISON_SEQUENCES_PER_SPECIES = 3  # จำนวนซีเควนซ์ต่อสปีชีส์สำหรับเปรียบเทียบ

# ตั้งค่าประเภทของซีเควนซ์ที่ต้องการ
SEARCH_TERMS = [
    "COI",
    "COXI",
    "COX1",
    "Cytochrome c Oxidase subunit I", 
]

# =============================================================================

def read_species_list(file_path):
    """อ่านรายชื่อสปีชีส์จากไฟล์"""
    try:
        with open(file_path, "r", encoding='utf-8') as f:
            species_list = [line.strip() for line in f if line.strip()]
        return species_list
    except FileNotFoundError:
        print(f"ไม่พบไฟล์: {file_path}")
        return []

def search_sequences(species, max_sequences=None, search_terms=None):
    """ค้นหาซีเควนซ์สำหรับสปีชีส์หนึ่ง"""
    if search_terms is None:
        search_terms = SEARCH_TERMS
    
    all_ids = []
    
    for term in search_terms:
        search_query = f"{species}[Organism] AND {term}"
        try:
            handle = Entrez.esearch(
                db="nucleotide", 
                term=search_query, 
                retmax=max_sequences if max_sequences else 100
            )
            record = Entrez.read(handle)
            handle.close()
            
            ids = record["IdList"]
            all_ids.extend(ids)
            
            print(f"  - พบ {len(ids)} ซีเควนซ์จาก '{term}'")
            
            if max_sequences and len(all_ids) >= max_sequences:
                break
                
        except Exception as e:
            print(f"  - ข้อผิดพลาดในการค้นหา '{term}': {e}")
            continue
    
    # ลบซีเควนซ์ที่ซ้ำกัน
    unique_ids = list(dict.fromkeys(all_ids))
    
    # จำกัดจำนวนตามที่ต้องการ
    if max_sequences:
        unique_ids = unique_ids[:max_sequences]
    
    return unique_ids

def fetch_sequence_metadata(seq_id):
    """ดึงข้อมูล metadata ของซีเควนซ์หนึ่ง"""
    try:
        # ดึงข้อมูล GenBank
        handle = Entrez.efetch(db="nucleotide", id=seq_id, rettype="gb", retmode="text")
        genbank_record = SeqIO.read(handle, "genbank")
        handle.close()
        
        # ดึงข้อมูล FASTA
        handle = Entrez.efetch(db="nucleotide", id=seq_id, rettype="fasta", retmode="text")
        fasta_record = SeqIO.read(handle, "fasta")
        handle.close()
        
        # สร้าง metadata
        metadata = {
            'Accession': genbank_record.id,
            'Title': genbank_record.description,
            'Organism': '',
            'Length': len(genbank_record.seq),
            'Country': '',
            'Geo_location': '',
            'Source_info': '',
            'Collection_date': '',
            'Isolation_source': '',
            'Strain': '',
            'Authors': '',
            'Journal': ''
        }
        
        # ดึงข้อมูลจาก source feature
        for feature in genbank_record.features:
            if feature.type == "source":
                qualifiers = feature.qualifiers
                
                if 'organism' in qualifiers:
                    metadata['Organism'] = qualifiers['organism'][0]
                
                if 'country' in qualifiers:
                    metadata['Country'] = qualifiers['country'][0]
                
                if 'geo_loc_name' in qualifiers:
                    metadata['Geo_location'] = qualifiers['geo_loc_name'][0]
                
                # รวบรวมข้อมูล source
                source_parts = []
                for key in ['host', 'tissue_type', 'dev_stage']:
                    if key in qualifiers:
                        source_parts.append(f"{key.replace('_', ' ').title()}: {qualifiers[key][0]}")
                if 'environmental_sample' in qualifiers:
                    source_parts.append("Environmental sample")
                
                metadata['Source_info'] = "; ".join(source_parts)
                
                if 'collection_date' in qualifiers:
                    metadata['Collection_date'] = qualifiers['collection_date'][0]
                
                if 'isolation_source' in qualifiers:
                    metadata['Isolation_source'] = qualifiers['isolation_source'][0]
                
                if 'strain' in qualifiers:
                    metadata['Strain'] = qualifiers['strain'][0]
                elif 'isolate' in qualifiers:
                    metadata['Strain'] = qualifiers['isolate'][0]
                
                break
        
        # ดึงข้อมูล references
        if genbank_record.annotations.get('references'):
            ref = genbank_record.annotations['references'][0]
            metadata['Authors'] = ref.authors if ref.authors else ''
            metadata['Journal'] = ref.journal if ref.journal else ''
        
        return fasta_record, metadata
        
    except Exception as e:
        print(f"    - ข้อผิดพลาดในการดึงข้อมูล {seq_id}: {e}")
        return None, None

def main():
    """ฟังก์ชันหลัก"""
    print("=== NCBI Sequence Downloader for Primer Design ===")
    print(f"สปีชีส์หลัก: {TARGET_SPECIES}")
    print(f"จำนวนซีเควนซ์ต่อสปีชีส์เปรียบเทียบ: {COMPARISON_SEQUENCES_PER_SPECIES}")
    print("=" * 50)
    
    # อ่านรายชื่อสปีชีส์ทั้งหมด
    all_species = read_species_list(FISH_LIST_FILE)
    if not all_species:
        print("ไม่พบรายชื่อสปีชีส์")
        return
    
    # แยกสปีชีส์หลักและสปีชีส์เปรียบเทียบ
    comparison_species = [s for s in all_species if s not in TARGET_SPECIES]
    
    print(f"สปีชีส์ทั้งหมด: {len(all_species)}")
    print(f"สปีชีส์หลัก: {len(TARGET_SPECIES)}")
    print(f"สปีชีส์เปรียบเทียบ: {len(comparison_species)}")
    print()
    
    # เตรียม CSV headers
    csv_headers = [
        'Species', 'Sequence_Type', 'Accession', 'Title', 'Organism', 'Length', 
        'Country', 'Geo_location', 'Source_info', 'Collection_date', 
        'Isolation_source', 'Strain', 'Authors', 'Journal'
    ]
    
    metadata_list = []
    sequence_count = defaultdict(int)
    
    # เปิดไฟล์ FASTA
    with open(OUTPUT_FASTA, "w") as fasta_handle:
        
        # ประมวลผลสปีชีส์หลัก (ดึงทั้งหมด)
        for species in TARGET_SPECIES:
            print(f"\n🎯 ประมวลผลสปีชีส์หลัก: {species}")
            
            sequence_ids = search_sequences(species, max_sequences=None)
            
            if not sequence_ids:
                print(f"  ไม่พบซีเควนซ์สำหรับ {species}")
                continue
            
            print(f"  พบ {len(sequence_ids)} ซีเควนซ์ทั้งหมด")
            
            for i, seq_id in enumerate(sequence_ids, 1):
                print(f"  ดึงข้อมูลซีเควนซ์ที่ {i}/{len(sequence_ids)}: {seq_id}")
                
                fasta_record, metadata = fetch_sequence_metadata(seq_id)
                
                if fasta_record and metadata:
                    # เขียน FASTA
                    SeqIO.write(fasta_record, fasta_handle, "fasta")
                    
                    # เพิ่ม metadata
                    metadata['Species'] = species
                    metadata['Sequence_Type'] = 'Target'
                    metadata_list.append(metadata)
                    sequence_count[species] += 1
                    
                    print(f"    ✅ {metadata['Accession']} - {metadata['Length']:,} bp")
                else:
                    print(f"    ❌ ไม่สามารถดึงข้อมูลได้")
                
                time.sleep(0.1)
        
        # ประมวลผลสปีชีส์เปรียบเทียบ
        for species in comparison_species:
            print(f"\n📊 ประมวลผลสปีชีส์เปรียบเทียบ: {species}")
            
            sequence_ids = search_sequences(species, max_sequences=COMPARISON_SEQUENCES_PER_SPECIES)
            
            if not sequence_ids:
                print(f"  ไม่พบซีเควนซ์สำหรับ {species}")
                continue
            
            print(f"  จะดึง {len(sequence_ids)} ซีเควนซ์")
            
            for i, seq_id in enumerate(sequence_ids, 1):
                print(f"  ดึงข้อมูลซีเควนซ์ที่ {i}/{len(sequence_ids)}: {seq_id}")
                
                fasta_record, metadata = fetch_sequence_metadata(seq_id)
                
                if fasta_record and metadata:
                    # เขียน FASTA
                    SeqIO.write(fasta_record, fasta_handle, "fasta")
                    
                    # เพิ่ม metadata
                    metadata['Species'] = species
                    metadata['Sequence_Type'] = 'Comparison'
                    metadata_list.append(metadata)
                    sequence_count[species] += 1
                    
                    print(f"    ✅ {metadata['Accession']} - {metadata['Length']:,} bp")
                else:
                    print(f"    ❌ ไม่สามารถดึงข้อมูลได้")
                
                time.sleep(0.1)
    
    # เขียน metadata ลง CSV
    with open(OUTPUT_METADATA, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=csv_headers)
        writer.writeheader()
        writer.writerows(metadata_list)
    
    # สรุปผลลัพธ์
    print("\n" + "=" * 50)
    print("🎉 เสร็จสิ้น!")
    print(f"📁 FASTA sequences: {OUTPUT_FASTA}")
    print(f"📊 Metadata: {OUTPUT_METADATA}")
    print(f"📈 จำนวนซีเควนซ์ทั้งหมด: {len(metadata_list)}")
    
    print(f"\n📋 สรุปตามสปีชีส์:")
    for species, count in sequence_count.items():
        species_type = "🎯 หลัก" if species in TARGET_SPECIES else "📊 เปรียบเทียบ"
        print(f"  {species_type} {species}: {count} ซีเควนซ์")
    
    # สรุปประเทศ
    countries = {}
    for metadata in metadata_list:
        if metadata['Country']:
            country = metadata['Country'].split(':')[0].strip()
            countries[country] = countries.get(country, 0) + 1
        elif metadata['Geo_location']:
            country = metadata['Geo_location'].split(':')[0].strip()
            countries[country] = countries.get(country, 0) + 1
    
    if countries:
        print(f"\n🌍 ประเทศที่พบ:")
        for country, count in sorted(countries.items()):
            print(f"  {country}: {count} ซีเควนซ์")

if __name__ == "__main__":
    main()

=== NCBI Sequence Downloader for Primer Design ===
สปีชีส์หลัก: ['oreochromis niloticus']
จำนวนซีเควนซ์ต่อสปีชีส์เปรียบเทียบ: 3
สปีชีส์ทั้งหมด: 843
สปีชีส์หลัก: 1
สปีชีส์เปรียบเทียบ: 843


🎯 ประมวลผลสปีชีส์หลัก: oreochromis niloticus
  - พบ 100 ซีเควนซ์จาก 'COI'
  - พบ 0 ซีเควนซ์จาก 'COXI'
  - พบ 100 ซีเควนซ์จาก 'COX1'
  - พบ 100 ซีเควนซ์จาก 'Cytochrome c Oxidase subunit I'
  พบ 200 ซีเควนซ์ทั้งหมด
  ดึงข้อมูลซีเควนซ์ที่ 1/200: 2814137097
    ✅ PQ313152.1 - 335 bp
  ดึงข้อมูลซีเควนซ์ที่ 2/200: 2814137095
    ✅ PQ313151.1 - 335 bp
  ดึงข้อมูลซีเควนซ์ที่ 3/200: 2814137093
    ✅ PQ313150.1 - 335 bp
  ดึงข้อมูลซีเควนซ์ที่ 4/200: 2814137091
    ✅ PQ313149.1 - 335 bp
  ดึงข้อมูลซีเควนซ์ที่ 5/200: 2814137089
    ✅ PQ313148.1 - 335 bp
  ดึงข้อมูลซีเควนซ์ที่ 6/200: 2814137087
    ✅ PQ313147.1 - 335 bp
  ดึงข้อมูลซีเควนซ์ที่ 7/200: 2814137085
    ✅ PQ313146.1 - 335 bp
  ดึงข้อมูลซีเควนซ์ที่ 8/200: 2814137083
    ✅ PQ313145.1 - 335 bp
  ดึงข้อมูลซีเควนซ์ที่ 9/200: 2797107365
    ✅ PQ272509.1 - 65